<h1>Objetivo</h1>
<h5><b>Avaliação:</b></h5><p>Para avaliar a performance da previsão do timestamp datahora, será usada a métrica RMSE. Para avaliar a performance da previsão de latitude e longitude, será usada a métrica Mean Haversine Distance</p>

<h1>Metodologia</h1>
<p>
<h3>Previsão datahora:</h3>
1 - Divisão de prepared_data_hist em treino e teste:

Dividir prepared_data_hist em conjunto de treino (dados mais antigos) e conjunto de teste (dados mais recentes) usando a coluna datahora_converted.
Criar X removendo a coluna datahora e y contendo apenas datahora.
Realizar validação cruzada, treinamento, teste e criação do modelo preditivo para prever datahora.

2 - Criação do DataFrame previsto_treino_data_datahora:

Fazer o merge de prepared_data_hist (sem a coluna datahora) com intermediate_treino_data_lat_long usando as colunas ordem, linha, latitude, longitude.
Adicionar a coluna id ao DataFrame resultante.

3 - Aplicação do modelo preditivo no DataFrame previsto_treino_data_datahora:

Remover a coluna id temporariamente para evitar conflitos.
Prever os valores de datahora usando o modelo criado no passo 1.
Adicionar os valores previstos de datahora como uma nova coluna.
Recolocar a coluna id.

4 - Preparação do DataFrame final após a predição:

Garantir que o DataFrame contém todas as colunas originais de prepared_data_hist (exceto datahora) mais a coluna datahora com os valores previstos e a coluna id.

5 - Criação do arquivo submissao_datahora:

Realizar o merge do DataFrame do passo 4 com intermediate_resposta_data_lat_long usando a coluna id.
Gerar o arquivo submissao_datahora contendo as colunas id, latitude, longitude, e datahora.

<h3>Previsão de latitude e longitude:</h3>

6 - Divisão de prepared_data_hist em treino e teste:

Dividir prepared_data_hist em conjunto de treino (dados mais antigos) e conjunto de teste (dados mais recentes) usando a coluna datahora_converted.
Criar X removendo as colunas latitude e longitude e y contendo apenas latitude e longitude.
Realizar validação cruzada, treinamento, teste e criação do modelo preditivo para prever latitude e longitude.

7 - Criação do DataFrame previsto_treino_data_lat_long:

Fazer o merge de prepared_data_hist (sem as colunas latitude e longitude) com intermediate_treino_data_datahora usando as colunas ordem, linha, datahora.
Adicionar a coluna id ao DataFrame resultante.

8 - Aplicação do modelo preditivo no DataFrame previsto_treino_data_lat_long:

Remover a coluna id temporariamente para evitar conflitos.
Prever os valores de latitude e longitude usando o modelo criado no passo 6.
Adicionar os valores previstos de latitude e longitude como novas colunas.
Recolocar a coluna id.

9 - Preparação do DataFrame final após a predição:

Garantir que o DataFrame contém todas as colunas originais de prepared_data_hist (exceto latitude e longitude) mais as colunas latitude e longitude com os valores previstos e a coluna id.

10 - Criação do arquivo submissao_lat_long:

Realizar o merge do DataFrame do passo 9 com intermediate_resposta_data_datahora usando a coluna id.
Gerar o arquivo submissao_lat_long contendo as colunas id, datahora, latitude, e longitude.
</p>

In [ ]:
import os
import zipfile
from glob import glob
import json
import cudf
import pandas as pd
from tqdm import tqdm
from cuml.cluster import DBSCAN
from memory_profiler import memory_usage
import numpy as np
import torch
import gc
import time

In [ ]:
# Caminhos das pastas
base_dir = os.getcwd()
dados_dir = os.path.join(base_dir, 'dados')
intermediarios_dir = os.path.join(base_dir, 'intermediarios')
processados_dir = os.path.join(base_dir, 'processados')

# Criar diretórios se não existirem
os.makedirs(intermediarios_dir, exist_ok=True)
os.makedirs(processados_dir, exist_ok=True)

# Lista global para armazenar os DataFrames processados
final_df_list = []

# Dicionário para armazenar o sentido atual de cada ônibus
sentidos_atual = {}

In [ ]:
def extract_and_process_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file in tqdm(zip_ref.namelist(), desc=f'Processing {os.path.basename(zip_path)}'):
            if file.endswith('.json'):
                zip_ref.extract(file, extract_to)
                json_path = os.path.join(extract_to, file)
                process_json(json_path)
                os.remove(json_path)

def process_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    df = cudf.DataFrame.from_pandas(pd.DataFrame(data))
    original_size = df.memory_usage(deep=True).sum()

    linhas_validas = [483, 864, 639, 3, 309, 774, 629, 371, 397, 100, 838, 315, 624, 388, 918, 665, 328, 497, 878, 355, 138, 606, 457, 550, 803, 917, 638, 2336, 399, 298, 867, 553, 565, 422, 756, 186012003, 292, 554, 634, 232, 415, 2803, 324, 852, 557, 759, 343, 779, 905, 108]
    df = df[df['linha'].astype(str).isin(map(str, linhas_validas))]

    #df['datahora'] = cudf.to_datetime(df['datahora'].astype('int64'), unit='ms')
    df['datahoraenvio'] = cudf.to_datetime(df['datahoraenvio'].astype('int64'), unit='ms')
    df['datahora_converted'] = cudf.to_datetime(df['datahora'].astype('int64'), unit='ms')

    df = df[(df['datahora_converted'].dt.hour >= 8) & (df['datahora_converted'].dt.hour < 22)]

    df['latitude'] = df['latitude'].str.replace(',', '.').astype('float32')
    df['longitude'] = df['longitude'].str.replace(',', '.').astype('float32')

    while df['latitude'].isnull().any() or df['longitude'].isnull().any():
        df['latitude'] = df['latitude'].interpolate()
        df['longitude'] = df['longitude'].interpolate()
        df['latitude'] = df['latitude'].fillna(method='ffill').fillna(method='bfill')
        df['longitude'] = df['longitude'].fillna(method='ffill').fillna(method='bfill')

    problematic_values = df[~df['velocidade'].astype(str).str.isnumeric()]
    if not problematic_values.empty:
        print(f"Problematic values in 'velocidade':\n{problematic_values[['velocidade', 'velocidade']].head()}")

    df['velocidade'] = df['velocidade'].astype('int32')

    df = df[(df['velocidade'] >= 0) & (df['velocidade'] <= 250)]
    df = df[df['latitude'].between(-90, 90) & df['longitude'].between(-180, 180)]

    reduced_size = df.memory_usage(deep=True).sum()

    final_df_list.append(df)

def save_intermediate_data(df, filename):
    intermediate_file_path = os.path.join(intermediarios_dir, filename)
    df.to_parquet(intermediate_file_path, index=False)
    print(f'Data saved to {intermediate_file_path}')

def load_or_process_data():
    global final_df_list
    final_df_list = []

    intermediate_file_path = os.path.join(intermediarios_dir, 'intermediate_data.parquet')

    if os.path.exists(intermediate_file_path):
        print("Intermediate file found. Loading data from intermediate file.")
        final_df = cudf.read_parquet(intermediate_file_path)
    else:
        print("Intermediate file not found. Processing JSON files.")
        zip_files = glob(os.path.join(dados_dir, '*.zip'))

        for zip_file in tqdm(zip_files, desc='Overall Progress'):
            extract_and_process_zip(zip_file, intermediarios_dir)

        # Concatenar todos os DataFrames processados
        final_df = cudf.concat(final_df_list, ignore_index=True)
        save_intermediate_data(final_df, 'intermediate_data.parquet')

    return final_df

def identify_garage_and_endpoints(df, epsilon=0.001, chunk_size=10000):
    if df['datahora_converted'].dtype == 'object':
        df['datahora_converted'] = cudf.to_datetime(df['datahora_converted'])

    grouped = df.groupby(['linha'])

    garage_routes = []
    endpoints = []

    for linha, group in grouped:
        group = group.sort_values(by='datahora_converted')

        ordens_unicas = group['ordem'].unique().to_pandas()
        for ordem in ordens_unicas:
            group_ordem = group[group['ordem'] == ordem].copy()

            # Identificar pontos de garagem baseando-se em desvios significativos da média
            mean_lat = group_ordem['latitude'].mean()
            mean_lon = group_ordem['longitude'].mean()

            diff_lat = (group_ordem['latitude'] - mean_lat).abs()
            diff_lon = (group_ordem['longitude'] - mean_lon).abs()

            garage_route = group_ordem[(diff_lat > epsilon) | (diff_lon > epsilon)]
            if not garage_route.empty:
                garage_routes.append(garage_route)

            # Identificar pontos finais baseando-se em paradas de 10 a 30 minutos
            group_ordem['time_diff'] = group_ordem['datahora_converted'].diff().astype('timedelta64[s]').fillna(0).astype('int32')
            endpoint = group_ordem[(group_ordem['time_diff'] >= 600) & (group_ordem['time_diff'] <= 1800) & (group_ordem['velocidade'] == 0)]
            if not endpoint.empty:
                endpoints.append(endpoint)

    if len(garage_routes) > 0:
        garage_routes = cudf.concat(garage_routes)
    else:
        garage_routes = cudf.DataFrame()

    if len(endpoints) > 0:
        endpoints = cudf.concat(endpoints)
    else:
        endpoints = cudf.DataFrame()

    def is_nearby_dbscan(df, points, epsilon, chunk_size):
        clustering = DBSCAN(eps=epsilon, min_samples=1)
        print(f"Running DBSCAN on {len(points)} points with epsilon={epsilon}")

        num_rows = df.shape[0]
        distances = np.empty(num_rows)
        
        for start in range(0, num_rows, chunk_size):
            end = min(start + chunk_size, num_rows)
            chunk = df[['latitude', 'longitude']].iloc[start:end]
            clustering.fit(chunk.to_pandas().values)
            labels = clustering.labels_
            nearest_points = chunk.to_pandas().values[labels == 0]

            chunk_distances = ((chunk.to_pandas().values[:, None] - nearest_points) ** 2).sum(axis=2) ** 0.5
            distances[start:end] = chunk_distances.min(axis=1)

            # Limpa a memória da GPU após cada batch
            gc.collect()
            torch.cuda.empty_cache()
        
        return cudf.Series(distances < epsilon)

    if not endpoints.empty:
        endpoint_points = endpoints[['latitude', 'longitude']]
        df['ponto_final'] = is_nearby_dbscan(df, endpoint_points, epsilon, chunk_size)
    else:
        df['ponto_final'] = False

    if not garage_routes.empty:
        garage_points = garage_routes[['latitude', 'longitude']]
        df['garagem'] = is_nearby_dbscan(df, garage_points, epsilon, chunk_size)
    else:
        df['garagem'] = False

    return df

def update_bus_direction(df):
    global sentidos_atual

    # Converte o DataFrame para pandas
    df = df.to_pandas()

    # Ordena o DataFrame pelas colunas 'ordem' e 'datahora_converted'
    df = df.sort_values(by=['ordem', 'datahora_converted'])

    # Inicializa a coluna 'sentido' como 'indo'
    df['sentido'] = 'indo'

    # Iteração sobre cada ônibus único
    ordens_unicas = df['ordem'].unique()
    for ordem in ordens_unicas:
        df_ordem = df[df['ordem'] == ordem].copy()

        sentido_atual = 'indo'
        sentidos = []

        for ponto_final in df_ordem['ponto_final']:
            if ponto_final:
                sentido_atual = 'voltando' if sentido_atual == 'indo' else 'indo'
            sentidos.append(sentido_atual)

        df.loc[df['ordem'] == ordem, 'sentido'] = sentidos

    # Converte de volta para cudf
    return cudf.DataFrame.from_pandas(df)

In [ ]:
def main():
    start_time = time.time()
    mem_usage_before = memory_usage()[0]

    final_df = load_or_process_data()

    mem_usage_after_load = memory_usage()[0]
    load_time = time.time()

    intermediate_path_after_loading = os.path.join(intermediarios_dir, 'data_after_loading.parquet')
    if not os.path.exists(intermediate_path_after_loading):
        save_intermediate_data(final_df, 'data_after_loading.parquet')

    final_df = identify_garage_and_endpoints(final_df)

    intermediate_path_after_processing = os.path.join(intermediarios_dir, 'data_after_garage_and_endpoints.parquet')
    if not os.path.exists(intermediate_path_after_processing):
        save_intermediate_data(final_df, 'data_after_garage_and_endpoints.parquet')

    mem_usage_after_process = memory_usage()[0]
    process_time = time.time()

    #final_df['datahora'] = cudf.to_datetime(final_df['datahora'])
    final_df['datahoraenvio'] = cudf.to_datetime(final_df['datahoraenvio'])
    #final_df['datahora_converted'] = cudf.to_datetime(final_df['datahora'])
    final_df['velocidade'] = final_df['velocidade'].astype('uint8')

    # Atualizar o sentido dos ônibus
    final_df = update_bus_direction(final_df)

    # Salvar os dados intermediários após atualizar o sentido dos ônibus
    intermediate_path_after_direction_update = os.path.join(intermediarios_dir, 'data_after_direction_update.parquet')
    if not os.path.exists(intermediate_path_after_direction_update):
        save_intermediate_data(final_df, 'data_after_direction_update.parquet')

    save_time = time.time()
    final_df.to_csv(os.path.join(processados_dir, 'processed_data.csv'), index=False, chunksize=10000)
    final_df.to_parquet(os.path.join(processados_dir, 'processed_data.parquet'), index=False)
    end_time = time.time()

    print(f'Tamanho final do DataFrame: {final_df.shape[0]}')
    print(f'Memory usage before loading data: {mem_usage_before} MiB')
    print(f'Memory usage after loading data: {mem_usage_after_load} MiB')
    print(f'Memory usage after processing data: {mem_usage_after_process} MiB')
    print(f'Time to load data: {load_time - start_time:.2f} seconds')
    print(f'Time to process data: {process_time - load_time:.2f} seconds')
    print(f'Time to save data: {end_time - save_time:.2f} seconds')
    print(f'Total execution time: {end_time - start_time:.2f} seconds')

In [ ]:
if __name__ == "__main__":
    main()

<h1>EDA</h1>

In [ ]:
final_processed_parquet_df = pd.read_parquet(os.path.join(processados_dir, 'processed_data.parquet'))

In [ ]:
final_processed_parquet_df.shape

In [ ]:
final_processed_parquet_df.head()

In [ ]:
final_processed_parquet_df.ponto_final.value_counts()

<h5>Verificando tipo de dados</h5>

In [ ]:
final_processed_parquet_df.dtypes

<h5>Análise Estatística</h5>

In [ ]:
final_processed_parquet_df.describe()

In [ ]:
final_processed_parquet_df.velocidade.max()

In [ ]:
import os
import pandas as pd
import cudf
import numpy as np
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates
from sklearn.model_selection import TimeSeriesSplit
import gc

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
class EDA:
    def __init__(self, file_path):
        """
        Inicializa a classe com o caminho do arquivo de dados processados.
        """
        self.file_path = file_path  

    def load_data(self, use_cudf=False):
        """
        Carrega os dados do arquivo Parquet usando pandas ou cudf.
        """
        try:
            if use_cudf:
                return cudf.read_parquet(self.file_path)  # Carrega o DataFrame usando cudf
            else:
                return pd.read_parquet(self.file_path)  # Carrega o DataFrame usando pandas
        except Exception as e:
            print(f"Erro ao carregar os dados: {e}")
            return None

    def initial_inspection(self):
        """
        Realiza uma inspeção inicial dos dados, visualizando as primeiras linhas e um resumo estatístico.
        """
        df = self.load_data(use_cudf=False)
        if df is not None:
            print(df.describe())  # Exibe um resumo estatístico dos dados
            del df  # Libera a memória
            gc.collect()  # Coleta o lixo

    def analyze_statistics(self):
        """
        Analisa as estatísticas dos dados, incluindo a distribuição das velocidades e a contagem de registros por linha e sentido.
        """
        df = self.load_data(use_cudf=False)
        if df is not None:
            # Plota a distribuição das velocidades
            df['velocidade'].hist(bins=50)  # Plota um histograma
            plt.title('Distribuição de Velocidades')
            plt.xlabel('Velocidade')
            plt.ylabel('Frequência')
            plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
            plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo x
            plt.show()

            # Conta registros por linha e sentido e exibe os resultados
            #print(df.groupby(['linha', 'sentido']).size().to_frame('count').reset_index())
            del df  # Libera a memória
            gc.collect()  # Coleta o lixo

    def analyze_temporal_patterns(self):
        """
        Analisa padrões temporais dos dados, incluindo o volume de dados por hora e padrões de movimentação ao longo do tempo.
        """
        df = self.load_data(use_cudf=False)
        if df is not None:
            # Extrai a hora do timestamp
            #df['datahora_converted'] = pd.to_datetime(df['datahora'])
            df['hora'] = df['datahora_converted'].dt.hour

            # Filtra as horas entre 8 e 22
            df = df[(df['hora'] >= 8) & (df['hora'] < 22)]

            # Conta registros por hora e plota o volume de dados por hora
            hourly_counts = df.groupby('hora').size().to_frame('count').reset_index()
            hourly_counts.plot(x='hora', y='count', kind='bar')
            plt.title('Volume de Dados por Hora')
            plt.xlabel('Hora do Dia')
            plt.ylabel('Contagem')
            plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
            plt.show()

            # Conta registros por dia ao longo do tempo e plota padrões de movimentação
            df['data'] = df['datahora_converted'].dt.date  # Extrai apenas a data
            daily_counts = df.groupby('data').size().to_frame('count').reset_index()
            daily_counts.plot(x='data', y='count', kind='line')
            plt.title('Padrões de Movimentação ao Longo do Tempo')
            plt.xlabel('Data')
            plt.ylabel('Contagem')
            plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Formata o eixo x para datas
            plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
            plt.xticks(rotation=45)
            plt.show()

            del df  # Libera a memória
            gc.collect()  # Coleta o lixo

    def analyze_geographical_patterns(self, linhas=None):
        """
        Analisa padrões geográficos dos dados, plotando um mapa de calor dos trajetos dos ônibus.
        """
        try:
            df_cudf = self.load_data(use_cudf=True)
            if df_cudf is not None:
                df_cudf['linha'] = df_cudf['linha'].astype(str)  # Garante que a coluna 'linha' seja string

                if linhas:
                    linhas = list(map(str, linhas))
                    for linha in linhas:
                        if linha not in df_cudf['linha'].to_pandas().tolist():
                            raise ValueError(f"Linha {linha} não encontrada nos dados.")

                    df_cudf = df_cudf[df_cudf['linha'].isin(linhas)]

                num_linhas = len(linhas) if linhas else len(df_cudf['linha'].unique())
                sample_fraction = min(1.0 / num_linhas, 0.01)  # Ajusta a fração de amostra de acordo com o número de linhas
                max_points = min(5000 * num_linhas, 50000)  # Ajusta o número máximo de pontos de acordo com o número de linhas

                # Define o centro do mapa baseado na média de latitude e longitude usando cudf
                map_center = [float(df_cudf['latitude'].mean()), float(df_cudf['longitude'].mean())]
                map_osm = folium.Map(location=map_center, zoom_start=12)  # Cria um mapa centrado na localização média

                # Amostra os dados antes de converter para pandas para evitar problemas de memória
                if len(df_cudf) > max_points:
                    df_sample = df_cudf.sample(frac=sample_fraction)
                    if len(df_sample) > max_points:
                        df_sample = df_sample.sample(n=max_points)
                else:
                    df_sample = df_cudf

                df_pandas = df_sample.to_pandas()

                # Dados para o mapa de calor
                heat_data = [[row['latitude'], row['longitude']] for index, row in df_pandas[['latitude', 'longitude']].iterrows()]
                HeatMap(heat_data).add_to(map_osm)  # Adiciona o mapa de calor ao mapa

                title = 'Mapa de Calor de Todas as Linhas'
                if linhas:
                    title = f'Mapa de Calor das Linhas: {", ".join(map(str, linhas))}'
                map_osm.get_root().html.add_child(folium.Element(f"<h1>{title}</h1>"))

                map_osm.save('bus_routes.html')  # Salva o mapa como um arquivo HTML
                print(f"Mapa de calor salvo como 'bus_routes.html' com {len(df_pandas)} pontos de amostra")
                
                del df_cudf  # Libera a memória
                del df_pandas  # Libera a memória
                gc.collect()  # Coleta o lixo
            else:
                print("Erro ao carregar os dados.")
        except ValueError as ve:
            print(ve)
        except Exception as e:
            print(f"Erro ao gerar o mapa de calor: {e}")

    def plot_routes_and_endpoints(self, linhas=None):
        """
        Plota os trajetos dos ônibus, locais de garagem e pontos finais com diferentes cores.
        """
        try:
            df_cudf = self.load_data(use_cudf=True)
            if df_cudf is not None:
                df_cudf['linha'] = df_cudf['linha'].astype(str)  # Garante que a coluna 'linha' seja string

                if linhas:
                    linhas = list(map(str, linhas))
                    for linha in linhas:
                        if linha not in df_cudf['linha'].to_pandas().tolist():
                            raise ValueError(f"Linha {linha} não encontrada nos dados.")

                    df_cudf = df_cudf[df_cudf['linha'].isin(linhas)]

                num_linhas = len(linhas) if linhas else len(df_cudf['linha'].unique())
                sample_fraction = min(1.0 / num_linhas, 0.01)  # Ajusta a fração de amostra de acordo com o número de linhas
                max_points = min(5000 * num_linhas, 50000)  # Ajusta o número máximo de pontos de acordo com o número de linhas

                # Define o centro do mapa baseado na média de latitude e longitude usando cudf
                map_center = [float(df_cudf['latitude'].mean()), float(df_cudf['longitude'].mean())]
                map_osm = folium.Map(location=map_center, zoom_start=12)  # Cria um mapa centrado na localização média

                # Amostra os dados antes de converter para pandas para evitar problemas de memória
                if len(df_cudf) > max_points:
                    df_sample = df_cudf.sample(frac=sample_fraction)
                    if len(df_sample) > max_points:
                        df_sample = df_sample.sample(n=max_points)
                else:
                    df_sample = df_cudf

                df_pandas = df_sample.to_pandas()

                # Plota os trajetos normais
                normal_routes = df_pandas[~df_pandas['garagem'] & ~df_pandas['ponto_final']]
                HeatMap([[row['latitude'], row['longitude']] for index, row in normal_routes[['latitude', 'longitude']].iterrows()],
                        name='Trajetos Normais', gradient={0.4: 'blue', 1: 'lightblue'}).add_to(map_osm)

                # Plota os locais de garagem
                garage_routes = df_pandas[df_pandas['garagem']]
                HeatMap([[row['latitude'], row['longitude']] for index, row in garage_routes[['latitude', 'longitude']].iterrows()],
                        name='Locais de Garagem', gradient={0.4: 'red', 1: 'darkred'}).add_to(map_osm)

                # Plota os pontos finais
                end_points = df_pandas[df_pandas['ponto_final']]
                HeatMap([[row['latitude'], row['longitude']] for index, row in end_points[['latitude', 'longitude']].iterrows()],
                        name='Pontos Finais', gradient={0.4: 'yellow', 1: 'gold'}).add_to(map_osm)

                title = 'Mapa de Trajetos, Locais de Garagem e Pontos Finais de Todas as Linhas'
                if linhas:
                    title = f'Mapa de Trajetos, Locais de Garagem e Pontos Finais das Linhas: {", ".join(map(str, linhas))}'
                map_osm.get_root().html.add_child(folium.Element(f"<h1>{title}</h1>"))

                map_osm.save('bus_routes_endpoints.html')  # Salva o mapa como um arquivo HTML
                print(f"Mapa de trajetos e pontos salvos como 'bus_routes_endpoints.html' com {len(df_pandas)} pontos de amostra")

                del df_cudf  # Libera a memória
                del df_pandas  # Libera a memória
                gc.collect()  # Coleta o lixo
            else:
                print("Erro ao carregar os dados.")
        except ValueError as ve:
            print(ve)
        except Exception as e:
            print(f"Erro ao gerar o mapa de trajetos e pontos: {e}")

    def analyze_endpoints_garage(self):
        """
        Analisa a distribuição dos pontos finais e pontos de garagem.
        """
        df = self.load_data(use_cudf=False)
        if df is not None:
            # Plota pontos finais
            df[df['ponto_final'] == True][['latitude', 'longitude']].plot(kind='scatter', x='longitude', y='latitude')
            plt.title('Distribuição de Pontos Finais')
            plt.show()

            # Plota pontos de garagem
            df[df['garagem'] == True][['latitude', 'longitude']].plot(kind='scatter', x='longitude', y='latitude')
            plt.title('Distribuição de Pontos de Garagem')
            plt.show()

            del df  # Libera a memória
            gc.collect()  # Coleta o lixo

    def analyze_routes(self):
        """
        Analisa os trajetos dos ônibus para cada linha.
        """
        df = self.load_data(use_cudf=False)
        if df is not None:
            linhas = df['linha'].unique()  # Obtém todas as linhas únicas
            num_linhas = len(linhas)
            num_cols = 3
            num_rows = (num_linhas + num_cols - 1) // num_cols  # Calcula o número de linhas para os subplots
            fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 5))
            axes = axes.flatten()

            for idx, linha in enumerate(linhas):
                trajeto = df[df['linha'] == linha]  # Filtra os dados pela linha
                ax = axes[idx]
                ax.plot(trajeto['longitude'], trajeto['latitude'], label=f'Linha {linha}')  # Plota o trajeto
                ax.set_title(f'Trajeto da Linha {linha}')
                ax.set_xlabel('Longitude')
                ax.set_ylabel('Latitude')
                ax.legend()
                ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
                ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo x

            # Remove subplots vazios
            for idx in range(len(linhas), len(axes)):
                fig.delaxes(axes[idx])

            plt.tight_layout()
            plt.show()

            del df  # Libera a memória
            gc.collect()  # Coleta o lixo

    def check_data_quality(self):
        """
        Verifica a qualidade dos dados, incluindo valores nulos, dados inconsistentes e outliers.
        """
        df = self.load_data(use_cudf=False)
        if df is not None:
            # Verifica valores nulos em cada coluna
            print("Verificando valores nulos...")
            print(df.isnull().sum())

            # Verifica valores não numéricos na coluna 'velocidade'
            print("Verificando dados inconsistentes...")
            problematic_values = df[~df['velocidade'].astype(str).str.isnumeric()]
            if not problematic_values.empty:
                print(f"Valores problemáticos em 'velocidade':\n{problematic_values[['velocidade', 'velocidade']].head()}")

            # Verifica outliers de velocidade
            print("Verificando outliers de velocidade [Velocidade acima de 150km/h]...")
            outliers = df[(df['velocidade'] < 0) | (df['velocidade'] > 150)]
            print(f"Outliers:\n{outliers}")

            del df  # Libera a memória
            gc.collect()  # Coleta o lixo

    def feature_engineering(self, df):
        """
        Cria novas features a partir das informações temporais.
        """
        print("Criando novas features...")
        #df['datahora_converted'] = pd.to_datetime(df['datahora'])
        df['dia_da_semana'] = df['datahora_converted'].dt.dayofweek  # Adiciona a feature 'dia_da_semana'
        df['hora'] = df['datahora_converted'].dt.hour  # Adiciona a feature 'hora'
        print("Novas features criadas: 'dia_da_semana', 'hora'")
        return df


    def run_all(self):
        """
        Executa todos os métodos de análise exploratória de dados em sequência.
        """
        self.initial_inspection()
        self.analyze_statistics()
        self.analyze_temporal_patterns()
        self.analyze_geographical_patterns()
        self.plot_routes_and_endpoints()
        self.analyze_endpoints_garage()
        self.analyze_routes()
        self.check_data_quality()


In [ ]:
base_dir = os.getcwd()
processados_dir = os.path.join(base_dir, 'processados')
eda = EDA(file_path=os.path.join(processados_dir, 'processed_data.parquet'))

# Carregar dados
#eda.load_data()

In [ ]:
# Realizar inspeção inicial
eda.initial_inspection()

In [ ]:
# Analisar estatísticas
eda.analyze_statistics()

In [ ]:
# Analisar padrões temporais
eda.analyze_temporal_patterns()

In [ ]:
# Analisar padrões geográficos
eda.analyze_geographical_patterns(linhas=["324", "3"])

In [ ]:
# Plotar trajetos e pontos
eda.plot_routes_and_endpoints(linhas=["324", "3"])

In [ ]:
# Analisar pontos finais e pontos de garagem
eda.analyze_endpoints_garage()

In [ ]:
# Analisar trajetos dos ônibus
eda.analyze_routes()


In [ ]:
# Verificar qualidade dos dados
eda.check_data_quality()

In [ ]:
# Realizar validação cruzada temporal
#eda.temporal_cross_validation()

<h1>Data Processing</h1>

<h5>Para não sobreescrever o dataframe processado final, irei criar um novo dataframe que sofrerá as devidas mudanças.</h5>

In [ ]:
import os
import pandas as pd
import cudf
import cupy as cp
import numpy as np
from cuml.cluster import DBSCAN
import gc
from tqdm import tqdm

# Suprime os avisos de SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [ ]:
class DataPreprocessing:
    def __init__(self, file_path, intermediarios_dir, processados_dir, chunk_size=1000000):
        self.file_path = file_path
        self.intermediarios_dir = intermediarios_dir
        self.processados_dir = processados_dir
        self.cleaned_file = os.path.join(intermediarios_dir, 'cleaned_data.parquet')
        self.prepared_file = os.path.join(processados_dir, 'prepared_data_hist.parquet')
        self.chunk_size = chunk_size

    def load_data(self):
        try:
            print("Carregando dados...")
            df = cudf.read_parquet(self.file_path)
            if 'datahora' not in df.columns:
                raise KeyError("A coluna 'datahora' não está presente nos dados carregados.")
            return df
        except Exception as e:
            print(f"Erro ao carregar os dados: {e}")
            return None

    def save_intermediate_data(self, df, filename):
        print(f"Salvando dados intermediários em '{filename}'...")
        df.to_parquet(filename)
        print(f"Dados salvos como '{filename}'")

    def remove_outliers(self, eps=0.001, min_samples=10):
        if os.path.exists(self.cleaned_file):
            print(f"Arquivo intermediário encontrado. Carregando dados de '{self.cleaned_file}'")
            df = cudf.read_parquet(self.cleaned_file)
            if 'datahora' not in df.columns:
                raise KeyError("A coluna 'datahora' não está presente nos dados intermediários carregados.")
            return df

        df = self.load_data()
        if df is not None:
            if 'datahora_converted' not in df.columns or 'linha' not in df.columns or 'ordem' not in df.columns:
                raise KeyError("As colunas necessárias não estão presentes nos dados.")

            lines = df['linha'].unique().to_pandas()
            cleaned_dfs = []

            print("Removendo outliers...")
            for linha in tqdm(lines, desc="Linhas"):
                line_data = df[df['linha'] == linha][['latitude', 'longitude', 'datahora_converted', 'linha', 'ordem', 'datahora']].copy()
                if len(line_data) < min_samples:
                    cleaned_dfs.append(line_data)
                    continue

                # Converte para pandas para DBSCAN
                line_data_pd = line_data[['latitude', 'longitude']].to_pandas()

                db = DBSCAN(eps=eps, min_samples=min_samples)
                labels = db.fit_predict(line_data_pd)

                # Adiciona os rótulos ao DataFrame
                line_data_pd['labels'] = labels

                # Filtra os pontos que não são outliers
                non_outliers = line_data_pd[line_data_pd['labels'] != -1]
                non_outliers = non_outliers.copy()  # Para evitar SettingWithCopyWarning
                non_outliers.loc[:, 'datahora_converted'] = line_data['datahora_converted'].to_pandas().loc[non_outliers.index]
                non_outliers.loc[:, 'linha'] = line_data['linha'].to_pandas().loc[non_outliers.index]
                non_outliers.loc[:, 'ordem'] = line_data['ordem'].to_pandas().loc[non_outliers.index]
                non_outliers.loc[:, 'datahora'] = line_data['datahora'].to_pandas().loc[non_outliers.index]

                cleaned_dfs.append(non_outliers)

            cleaned_df = cudf.DataFrame.from_pandas(pd.concat(cleaned_dfs, ignore_index=True))
            self.save_intermediate_data(cleaned_df, self.cleaned_file)

            if 'datahora' not in cleaned_df.columns:
                raise KeyError("A coluna 'datahora' foi removida durante a remoção de outliers.")

            return cleaned_df
        else:
            print("Erro ao carregar os dados para remoção de outliers.")
            return None

    def feature_engineering(self, df):
        print("Criando novas features...")

        if 'datahora_converted' not in df.columns or 'linha' not in df.columns or 'ordem' not in df.columns:
            raise KeyError("As colunas necessárias não estão presentes nos dados.")

        df = df.sort_values(by=['linha', 'ordem', 'datahora_converted'])

        df['dia_da_semana'] = df['datahora_converted'].dt.weekday
        df['hora'] = df['datahora_converted'].dt.hour
        df['diff_timestamp'] = df.groupby(['linha', 'ordem'])['datahora_converted'].diff().astype('int64') / 10**6

        df['latitude_diff'] = df.groupby(['linha', 'ordem'])['latitude'].diff().fillna(method='ffill')
        df['longitude_diff'] = df.groupby(['linha', 'ordem'])['longitude'].diff().fillna(method='ffill')

        df_pandas = df[['latitude', 'longitude', 'latitude_diff', 'longitude_diff']].to_pandas()
        lat1 = np.radians(df_pandas['latitude'])
        lon1 = np.radians(df_pandas['longitude'])
        lat2 = np.radians(df_pandas['latitude'] + df_pandas['latitude_diff'])
        lon2 = np.radians(df_pandas['longitude'] + df_pandas['longitude_diff'])

        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
        c = 2 * np.arcsin(np.sqrt(a))
        r = 6371
        df_pandas['distancia'] = c * r * 1000

        df['distancia'] = cudf.from_pandas(df_pandas['distancia'])

        print("Novas features criadas: 'dia_da_semana', 'hora', 'diff_timestamp', 'latitude_diff', 'longitude_diff', 'distancia'")

        if 'datahora' not in df.columns:
            raise KeyError("A coluna 'datahora' foi removida durante a engenharia de features.")

        return df

    def prepare_data_for_prediction(self):
        if os.path.exists(self.prepared_file):
            print(f"Arquivo intermediário encontrado. Carregando dados de '{self.prepared_file}'")
            df = cudf.read_parquet(self.prepared_file)
            if 'datahora' not in df.columns:
                raise KeyError("A coluna 'datahora' não está presente nos dados intermediários carregados.")
            return df

        df = self.remove_outliers()
        if df is not None:
            df = self.feature_engineering(df)
            self.save_intermediate_data(df, self.prepared_file)

            if 'datahora' not in df.columns:
                raise KeyError("A coluna 'datahora' foi removida durante o processamento.")
            return df
        else:
            print("Erro ao preparar os dados para previsão.")
            return None

In [ ]:
base_dir = os.getcwd()
intermediarios_dir = os.path.join(base_dir, 'intermediarios')
processados_dir = os.path.join(base_dir, 'processados')
preprocessing = DataPreprocessing(os.path.join(processados_dir, 'processed_data.parquet'), intermediarios_dir, processados_dir)

In [ ]:
# Remover outliers e preparar dados
prepared_data = preprocessing.prepare_data_for_prediction()

<h3>Pré processando dados de teste</h3>

In [ ]:
import os
import cudf
import pandas as pd
import zipfile
import json
from glob import glob
from tqdm import tqdm
import gc
from sklearn.cluster import DBSCAN
import warnings
# Suprimindo warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
class TestDataProcessor:
    def __init__(self, base_dir):
        self.base_dir = os.path.join(base_dir, 'dados_teste')
        self.intermediarios_teste_dir = os.path.join(self.base_dir, 'intermediarios_teste')
        self.processados_teste_dir = os.path.join(self.base_dir, 'processados_teste')
        self.chunk_size = 100000

        os.makedirs(self.intermediarios_teste_dir, exist_ok=True)
        os.makedirs(self.processados_teste_dir, exist_ok=True)

    def extract_zip_files(self, zip_files):
        for zip_file in tqdm(zip_files, desc="Extracting zip files"):
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_ref.extractall(self.intermediarios_teste_dir)

    def process_json(self, json_path, df_list):
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        df = cudf.DataFrame.from_pandas(pd.DataFrame(data))
        df_list.append(df)

    def process_and_concat_files(self, prefix, output_parquet):
        all_files = []
        for root, _, files in os.walk(self.intermediarios_teste_dir):
            for file in files:
                if file.startswith(prefix) and file.endswith('.json'):
                    all_files.append(os.path.join(root, file))
        
        final_df_list = []
        for json_file in tqdm(all_files, desc=f"Processing {prefix} files"):
            self.process_json(json_file, final_df_list)

        if final_df_list:
            final_df = cudf.concat(final_df_list, ignore_index=True)
            final_df.to_parquet(output_parquet)
            print(f"Saved concatenated {prefix} data to {output_parquet}")
        else:
            print(f"No {prefix} JSON files found.")
            final_df = None

        return final_df

    def process_recent_data_iteratively(self, prefix, output_csv):
        all_files = []
        for root, _, files in os.walk(self.intermediarios_teste_dir):
            for file in files:
                if file.startswith(prefix) and file.endswith('.json'):
                    all_files.append(os.path.join(root, file))

        final_df_list = []
        for json_file in tqdm(all_files, desc=f"Processing {prefix} files"):
            self.process_and_append_json(json_file, final_df_list, output_csv)

        if final_df_list:
            final_df = cudf.concat(final_df_list, ignore_index=True)
        else:
            final_df = None

        return final_df

    def process_and_append_json(self, json_path, df_list, output_csv):
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        df = cudf.DataFrame.from_pandas(pd.DataFrame(data))

        # Processando dados recentes
        linhas_validas = [483, 864, 639, 3, 309, 774, 629, 371, 397, 100, 838, 315, 624, 388, 918, 665, 328, 497, 878, 355, 138, 606, 457, 550, 803, 917, 638, 2336, 399, 298, 867, 553, 565, 422, 756, 186012003, 292, 554, 634, 232, 415, 2803, 324, 852, 557, 759, 343, 779, 905, 108]
        df = df[df['linha'].astype(str).isin(map(str, linhas_validas))]
        
        df['datahoraenvio'] = cudf.to_datetime(df['datahoraenvio'].astype('int64'), unit='ms')
        df['datahora_converted'] = cudf.to_datetime(df['datahora'].astype('int64'), unit='ms')
        df = df[(df['datahora_converted'].dt.hour >= 8) & (df['datahora_converted'].dt.hour < 22)]

        df['latitude'] = df['latitude'].str.replace(',', '.').astype('float32')
        df['longitude'] = df['longitude'].str.replace(',', '.').astype('float32')

        while df['latitude'].isnull().any() or df['longitude'].isnull().any():
            df['latitude'] = df['latitude'].interpolate().ffill().bfill()
            df['longitude'] = df['longitude'].interpolate().ffill().bfill()

        df['velocidade'] = df['velocidade'].astype('int32')
        df = df[(df['velocidade'] >= 0) & (df['velocidade'] <= 250)]
        df = df[df['latitude'].between(-90, 90) & df['longitude'].between(-180, 180)]

        df_list.append(df)

        # Salva intermediário iterativamente em CSV
        if os.path.exists(output_csv):
            df.to_pandas().to_csv(output_csv, mode='a', header=False, index=False)
        else:
            df.to_pandas().to_csv(output_csv, index=False)

    def run(self):
        zip_files = glob(os.path.join(self.base_dir, '*.zip'))
        
        # Debug: print o diretório atual e os arquivos encontrados
        print(f"Procurando arquivos zip em: {self.base_dir}")
        print(f"Arquivos zip encontrados: {zip_files}")

        if not zip_files:
            raise FileNotFoundError("No zip files found in the specified directory.")

        self.extract_zip_files(zip_files)

        treino_intermediate_parquet = os.path.join(self.intermediarios_teste_dir, 'intermediate_treino_data.parquet')
        resposta_intermediate_parquet = os.path.join(self.intermediarios_teste_dir, 'intermediate_resposta_data.parquet')
        recent_intermediate_csv = os.path.join(self.intermediarios_teste_dir, 'intermediate_recent_data.csv')
        recent_intermediate_parquet = os.path.join(self.intermediarios_teste_dir, 'intermediate_recent_data.parquet')
        recent_raw_intermediate_parquet = os.path.join(self.intermediarios_teste_dir, 'intermediate_recent_raw_data.parquet')

        # Concatenando arquivos de treino e resposta
        #treino_data_df = self.process_and_concat_files('treino', treino_intermediate_parquet)
        #resposta_data_df = self.process_and_concat_files('resposta', resposta_intermediate_parquet)

        # Processamento iterativo dos dados recentes
        if os.path.exists(recent_intermediate_csv):
            recent_data_df = cudf.read_csv(recent_intermediate_csv)
        else:
            recent_data_df = self.process_recent_data_iteratively('2024', recent_intermediate_csv)

        # Salvar como parquet após processamento iterativo
        if recent_data_df is not None:
            recent_data_df.to_parquet(recent_intermediate_parquet)

        # Salvar dados recentes sem processamento
        if os.path.exists(recent_intermediate_csv):
            recent_raw_data_df = cudf.read_csv(recent_intermediate_csv)
            recent_raw_data_df.to_parquet(recent_raw_intermediate_parquet)
            print(f"Saved raw recent data to {recent_raw_intermediate_parquet}")

        data_preprocessor = DataPreprocessing(
            file_path=recent_intermediate_parquet,
            intermediarios_dir=self.intermediarios_teste_dir,
            processados_dir=self.processados_teste_dir,
            chunk_size=self.chunk_size
        )

        data_preprocessor.prepare_data_for_prediction()

In [ ]:
if __name__ == "__main__":
    base_dir = os.getcwd()
    processor = TestDataProcessor(base_dir)
    processor.run()

<h5>Ajustar dataset de treino e de resposta</h5>

In [ ]:
import os
import cudf 

base_dir = os.getcwd()
dados_teste_dir = os.path.join(base_dir, 'dados_teste')
intermediarios_teste_dir = os.path.join(dados_teste_dir, 'intermediarios_teste')
processados_teste_dir = os.path.join(dados_teste_dir, 'processados_teste')

#Path for loading
path = os.path.join(intermediarios_teste_dir, "intermediate_treino_data.parquet")
path2 = os.path.join(intermediarios_teste_dir, "intermediate_resposta_data.parquet")

#Path for saving the data for not null datahora
path3_datahora = os.path.join(processados_teste_dir, "intermediate_treino_data_datahora.parquet")
path4_datahora = os.path.join(processados_teste_dir, "intermediate_resposta_data_datahora.parquet")

#Path for saving the data for not null lat/long
path3_lat_long = os.path.join(processados_teste_dir, "intermediate_treino_data_lat_long.parquet")
path4_lat_long = os.path.join(processados_teste_dir, "intermediate_resposta_data_lat_long.parquet")

treino = cudf.read_parquet(path)
resposta = cudf.read_parquet(path2)

In [ ]:
treino[treino['datahora'].isna() == False].head()

<h5>Criando 2 diferentes datasets. Um para ser usado para a previsão de latitude e 
longitude (possui a coluna datahora) e outro para a previsão de datahora (possui latitude e longitude)</h5>

In [ ]:
#Mudanças para o dataset de treino
treino_horadata = treino.copy()
treino_lat_long = treino.copy()

treino_horadata.drop(['latitude', 'longitude'], axis=1, inplace=True)
treino_horadata.dropna(subset=['datahora'],inplace=True)

treino_lat_long.drop(['datahora'], axis=1, inplace=True)
treino_lat_long.dropna(subset=['latitude', 'longitude'],inplace=True)

#Mudanças para o dataset de resposta
resposta_horadata = resposta.copy()
resposta_lat_long = resposta.copy()

resposta_horadata.drop(['latitude', 'longitude'], axis=1, inplace=True)
resposta_horadata.dropna(subset=['datahora'],inplace=True)

resposta_lat_long.drop(['datahora'], axis=1, inplace=True)
resposta_lat_long.dropna(subset=['latitude', 'longitude'],inplace=True)


In [ ]:
treino_horadata.to_parquet(path3_datahora)
treino_lat_long.to_parquet(path3_lat_long)

resposta_horadata.to_parquet(path4_datahora)
resposta_lat_long.to_parquet(path4_lat_long)

In [ ]:
#intermediate_resposta_data_datahora
resposta_horadata.head()

In [ ]:
#intermediate_resposta_data_lat_long
resposta_lat_long.head()

In [ ]:
treino_horadata.head()

In [ ]:
treino_lat_long.head()

In [ ]:
treino.shape

In [ ]:
#treino_horadata será usado para prever resposta_lat_long
treino_horadata.shape, resposta_lat_long.shape

In [ ]:
#treino_lat_long será usado para prever resposta_hora_data
treino_lat_long.shape, resposta_horadata.shape

<h1>Previsão de duas métricas diferentes: Datahora e Latitude/Longitude</h1>

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import joblib
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import optuna
from optuna.integration import LightGBMPruningCallback
from sklearn.model_selection import TimeSeriesSplit
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import logging

import warnings
warnings.filterwarnings('ignore')

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
def haversine_rmse(y_true, y_pred):
    R = 6371  # Radius of the Earth in kilometers
    lat1 = np.radians(y_true[:, 0])
    lon1 = np.radians(y_true[:, 1])
    lat2 = np.radians(y_pred[:, 0])
    lon2 = np.radians(y_pred[:, 1])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return np.sqrt(np.mean(distance**2))

def objective_xgb(trial, X, y, is_location):
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor'
    }
    model = XGBRegressor(**param)
    if is_location:
        model = MultiOutputRegressor(model)
    scores = []
    kfold = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        try:
            logging.info(f"X_test: {X_test.head()}")
            logging.info(f"y_test: {y_test.head()}")
            logging.info(f"Predictions: {preds[:5]}")
            
            if np.any(np.isnan(preds)):
                logging.error(f"NaN values found in predictions: {preds}")
                raise ValueError("NaN values found in predictions.")
            if not all(dtype == 'int' for dtype in y_test.dtypes) and not is_location:
                logging.error(f"y_test data type is not int: {y_test.dtypes}")
                raise ValueError(f"y_test data type is not int: {y_test.dtypes}")
            if is_location:
                score = haversine_rmse(y_test.values, preds)
            else:
                y_test_converted = y_test.values.astype(np.int64) // 10**9
                preds_converted = preds.astype(np.int64) // 10**9
                score = np.sqrt(mean_squared_error(y_test_converted, preds_converted))
            logging.info(f"Score: {score}")
            scores.append(score)
        except Exception as e:
            logging.error(f"Error in calculating score: {e}")
            raise e
    return np.mean(scores)

def objective_lgbm(trial, X, y, is_location):
    param = {
        'num_leaves': trial.suggest_int('num_leaves', 31, 127),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'device': 'cuda',
        'gpu_platform_id': 0,
        'gpu_device_id': 0
    }
    model = LGBMRegressor(**param)
    if is_location:
        model = MultiOutputRegressor(model)
    scores = []
    kfold = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        try:
            logging.info(f"X_test: {X_test.head()}")
            logging.info(f"y_test: {y_test.head()}")
            logging.info(f"Predictions: {preds[:5]}")
            
            if np.any(np.isnan(preds)):
                logging.error(f"NaN values found in predictions: {preds}")
                raise ValueError("NaN values found in predictions.")
            if not all(dtype == 'int' for dtype in y_test.dtypes) and not is_location:
                logging.error(f"y_test data type is not int: {y_test.dtypes}")
                raise ValueError(f"y_test data type is not int: {y_test.dtypes}")
            if is_location:
                score = haversine_rmse(y_test.values, preds)
            else:
                y_test_converted = y_test.values.astype(np.int64) // 10**9
                preds_converted = preds.astype(np.int64) // 10**9
                score = np.sqrt(mean_squared_error(y_test_converted, preds_converted))
            logging.info(f"Score: {score}")
            scores.append(score)
        except Exception as e:
            logging.error(f"Error in calculating score: {e}")
            raise e
    return np.mean(scores)

def run_optimized_models(df, target_cols, save_dir, is_location=False):
    X = df.drop(columns=target_cols)
    y = df[target_cols]

    model_definitions = {
        'XGBoost': XGBRegressor(tree_method='gpu_hist', predictor='gpu_predictor', gpu_id=0, n_jobs=-1),
        'LightGBM': LGBMRegressor(device='cuda', gpu_platform_id=0, gpu_device_id=0, force_row_wise=True, n_jobs=-1)
    }

    best_params = {}
    model_results = {}
    rmse_scores = {'XGBoost': [], 'LightGBM': []}

    for model_name, model in tqdm(model_definitions.items(), desc="Optimizing models"):
        model_path = os.path.join(save_dir, f"{model_name}_best_model.pkl")
        if os.path.exists(model_path):
            logging.info(f"Model {model_name} already exists. Loading from file.")
            best_model = joblib.load(model_path)
            model_results[model_name] = best_model
            continue

        if model_name == 'XGBoost':
            study = optuna.create_study(direction='minimize')
            for _ in tqdm(range(10), desc=f"Optuna Trials for {model_name}"):
                study.optimize(lambda trial: objective_xgb(trial, X, y, is_location), n_trials=1)
            best_params[model_name] = study.best_params
            best_params[model_name].pop('gpu_id', None)
            best_params[model_name].pop('n_jobs', None)
            best_model = XGBRegressor(**best_params[model_name], gpu_id=0, n_jobs=-1)
            if is_location:
                best_model = MultiOutputRegressor(best_model)
        else:
            study = optuna.create_study(direction='minimize')
            for _ in tqdm(range(10), desc=f"Optuna Trials for {model_name}"):
                study.optimize(lambda trial: objective_lgbm(trial, X, y, is_location), n_trials=1)
            best_params[model_name] = study.best_params
            best_params[model_name].pop('device', None)
            best_params[model_name].pop('gpu_platform_id', None)
            best_params[model_name].pop('gpu_device_id', None)
            best_model = LGBMRegressor(**best_params[model_name], device='cuda', gpu_platform_id=0, gpu_device_id=0, force_row_wise=True, n_jobs=-1)
            if is_location:
                best_model = MultiOutputRegressor(best_model)

        kfold = TimeSeriesSplit(n_splits=10)
        for train_index, test_index in kfold.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            best_model.fit(X_train, y_train)
            preds = best_model.predict(X_test)
            try:
                if np.any(np.isnan(preds)):
                    logging.error(f"NaN values found in predictions for {model_name}: {preds}")
                    raise ValueError("NaN values found in predictions.")
                if not all(dtype == 'int' for dtype in y_test.dtypes) and not is_location:
                    logging.error(f"y_test data type is not int for {model_name}: {y_test.dtypes}")
                    raise ValueError(f"y_test data type is not int for {model_name}: {y_test.dtypes}")
                if is_location:
                    score = haversine_rmse(y_test.values, preds)
                else:
                    y_test_converted = y_test.values.astype(np.int64) // 10**9
                    preds_converted = preds.astype(np.int64) // 10**9
                    score = np.sqrt(mean_squared_error(y_test_converted, preds_converted))
                rmse_scores[model_name].append(score)
            except Exception as e:
                logging.error(f"Error in calculating score for {model_name}: {e}")
                raise e

        joblib.dump(best_model, model_path)
        model_results[model_name] = best_model

    mean_rmse_scores = {model: np.mean(scores) for model, scores in rmse_scores.items() if scores}
    best_model_name = min(mean_rmse_scores, key=mean_rmse_scores.get) if mean_rmse_scores else None
    logging.info(f"Mean RMSE scores: {mean_rmse_scores}")
    logging.info(f"Best model: {best_model_name} with mean score: {mean_rmse_scores.get(best_model_name)}")
    
    return model_results, best_model_name

In [ ]:
def create_final_model(df, target_cols, best_model_name, best_params, save_dir, is_location=False):
    X = df.drop(columns=target_cols)
    y = df[target_cols]

    if best_model_name == 'XGBoost':
        best_params.pop('gpu_id', None)
        best_params.pop('n_jobs', None)
        best_model = XGBRegressor(**best_params, gpu_id=0, n_jobs=-1)
        if is_location:
            best_model = MultiOutputRegressor(best_model)
    else:
        best_params.pop('device', None)
        best_params.pop('gpu_platform_id', None)
        best_params.pop('gpu_device_id', None)
        best_model = LGBMRegressor(**best_params, device='cuda', gpu_platform_id=0, gpu_device_id=0, force_row_wise=True, n_jobs=-1)
        if is_location:
            best_model = MultiOutputRegressor(best_model)

    best_model.fit(X, y)
    joblib.dump(best_model, os.path.join(save_dir, f"final_{best_model_name}_model.pkl"))

    return best_model

In [ ]:
def prepare_and_submit(model, recent_data, submission_path, predict_cols):
    preds = model.predict(recent_data.drop(columns=predict_cols))
    for idx, col in enumerate(predict_cols):
        recent_data[col] = preds[:, idx] if preds.ndim > 1 else preds
    recent_data.to_parquet(submission_path, index=False)

def plot_differences(df_real, df_pred, columns, title, ylabel, output_path):
    differences = df_real[columns] - df_pred[columns]
    plt.figure(figsize=(10, 5))
    plt.plot(differences)
    plt.title(title)
    plt.xlabel('Sample Index')
    plt.ylabel(ylabel)
    plt.savefig(output_path)
    plt.close()


In [ ]:
def main():
    base_dir = os.getcwd()
    dados_teste_dir = os.path.join(base_dir, 'dados_teste', 'processados_teste')
    resultados_dir = os.path.join(dados_teste_dir, 'resultados')
    submissoes_dir = os.path.join(dados_teste_dir, 'submissoes')
    os.makedirs(resultados_dir, exist_ok=True)
    os.makedirs(submissoes_dir, exist_ok=True)

    arquivos = [
        'prepared_data_hist.parquet',
        'intermediate_treino_data_datahora.parquet',
        'intermediate_treino_data_lat_long.parquet',
        'intermediate_resposta_data_datahora.parquet',
        'intermediate_resposta_data_lat_long.parquet'
    ]

    arquivos_existentes = {arquivo: os.path.exists(os.path.join(dados_teste_dir, arquivo)) for arquivo in arquivos}

    if all(arquivos_existentes.values()):
        logging.info("Todos os arquivos necessários já existem. Carregando os dados...")
        df_hist = pd.read_parquet(os.path.join(dados_teste_dir, 'prepared_data_hist.parquet'))
        df_treino_datahora = pd.read_parquet(os.path.join(dados_teste_dir, 'intermediate_treino_data_datahora.parquet'))
        df_treino_lat_long = pd.read_parquet(os.path.join(dados_teste_dir, 'intermediate_treino_data_lat_long.parquet'))
        df_resposta_datahora = pd.read_parquet(os.path.join(dados_teste_dir, 'intermediate_resposta_data_datahora.parquet'))
        df_resposta_lat_long = pd.read_parquet(os.path.join(dados_teste_dir, 'intermediate_resposta_data_lat_long.parquet'))

        # Prever datahora baseado em latitude e longitude
        df_lat_long = df_hist[['latitude', 'longitude', 'datahora']]
        na_cols = df_lat_long.columns[df_lat_long.isna().any()].tolist()
        if na_cols:
            raise ValueError(f"NaNs found in data for 'Prever datahora': {na_cols}")

        df_lat_long['datahora'] = pd.to_datetime(df_lat_long['datahora']).view(np.int64) // 10**9
        df_treino_lat_long['datahora'] = pd.to_datetime(df_treino_lat_long['datahora']).view(np.int64) // 10**9

        model_results, best_model_name = run_optimized_models(df_lat_long.copy(), target_cols=['datahora'], save_dir=resultados_dir)
        best_model = create_final_model(df_lat_long.copy(), target_cols=['datahora'], best_model_name=best_model_name, best_params=model_results[best_model_name].get_params(), save_dir=resultados_dir)
        prepare_and_submit(best_model, df_treino_lat_long.copy(), os.path.join(resultados_dir, 'previsto_treino_data_datahora.parquet'), predict_cols=['datahora'])

        # Prever latitude e longitude baseado em datahora
        df_datahora = df_hist[['datahora', 'latitude', 'longitude']]
        na_cols = df_datahora.columns[df_datahora.isna().any()].tolist()
        if na_cols:
            raise ValueError(f"NaNs found in data for 'Prever latitude e longitude': {na_cols}")

        df_datahora['datahora'] = pd.to_datetime(df_datahora['datahora']).view(np.int64) // 10**9
        df_treino_datahora['datahora'] = pd.to_datetime(df_treino_datahora['datahora']).view(np.int64) // 10**9

        model_results, best_model_name = run_optimized_models(df_datahora.copy(), target_cols=['latitude', 'longitude'], save_dir=resultados_dir, is_location=True)
        best_model = create_final_model(df_datahora.copy(), target_cols=['latitude', 'longitude'], best_model_name=best_model_name, best_params=model_results[best_model_name].get_params(), save_dir=resultados_dir, is_location=True)
        prepare_and_submit(best_model, df_treino_datahora.copy(), os.path.join(resultados_dir, 'previsto_treino_data_lat_long.parquet'), predict_cols=['latitude', 'longitude'])

        # Criar submissão
        df_previsto_datahora = pd.read_parquet(os.path.join(resultados_dir, 'previsto_treino_data_datahora.parquet'))
        df_previsto_lat_long = pd.read_parquet(os.path.join(resultados_dir, 'previsto_treino_data_lat_long.parquet'))

        submissao_datahora = df_resposta_datahora.merge(df_previsto_datahora[['id', 'datahora']], on='id', how='left')
        submissao_datahora['datahora'] = pd.to_datetime(submissao_datahora['datahora'] * 10**9)
        submissao_datahora.to_parquet(os.path.join(submissoes_dir, 'submissao_datahora.parquet'), index=False)

        submissao_lat_long = df_resposta_lat_long.merge(df_previsto_lat_long[['id', 'latitude', 'longitude']], on='id', how='left')
        submissao_lat_long.to_parquet(os.path.join(submissoes_dir, 'submissao_lat_long.parquet'), index=False)

        # Plotar diferenças
        df_resposta_datahora['datahora'] = pd.to_datetime(df_resposta_datahora['datahora'])
        df_previsto_datahora['datahora'] = pd.to_datetime(df_previsto_datahora['datahora'] * 10**9)
        df_resposta_lat_long['datahora'] = pd.to_datetime(df_resposta_lat_long['datahora'])
        df_previsto_lat_long['datahora'] = pd.to_datetime(df_previsto_lat_long['datahora'] * 10**9)

        plot_differences(df_resposta_datahora, df_previsto_datahora, 'datahora', 'Diferença de Datahora (Previsto vs Real)', 'Diferença em Segundos', os.path.join(resultados_dir, 'diferenca_datahora.png'))
        plot_differences(df_resposta_lat_long, df_previsto_lat_long, ['latitude', 'longitude'], 'Diferença de Latitude e Longitude (Previsto vs Real)', 'Diferença em Graus', os.path.join(resultados_dir, 'diferenca_lat_long.png'))


In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
#TESTANDO ATÉ FUNCIONAR ABAIXO Mean Haversine Distance
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#import warnings
#warnings.filterwarnings('ignore', category=UserWarning)

#df['latitude'] = df['latitude'].str.replace(',', '.').astype('float32')
#df['longitude'] = df['longitude'].str.replace(',', '.').astype('float32')

In [1]:
import pandas as pd
import cudf
import numpy as np
import cupy as cp
import optuna
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
import logging
import joblib
import os
import gc
from tqdm import tqdm
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Directories
base_dir = os.getcwd()
dados_teste_dir = os.path.join(base_dir, 'dados_teste', 'processados_teste')
resultados_dir = os.path.join(dados_teste_dir, 'resultados')
submissoes_dir = os.path.join(dados_teste_dir, 'submissoes')
temp_dir = os.path.join(resultados_dir, 'temp')
os.makedirs(resultados_dir, exist_ok=True)
os.makedirs(submissoes_dir, exist_ok=True)
os.makedirs(temp_dir, exist_ok=True)

arquivos = [
    'prepared_data_hist.parquet',
    'intermediate_treino_data_datahora.parquet',
    'intermediate_treino_data_lat_long.parquet',
    'intermediate_resposta_data_datahora.parquet',
    'intermediate_resposta_data_lat_long.parquet'
]

def convert_dtypes_pandas(df):
    if "latitude" in df.columns and (df["latitude"].dtype == 'object' or df["latitude"].dtype.name == 'category'):
        df["latitude"] = df["latitude"].str.replace(',', '.').astype('float32')
    if "longitude" in df.columns and (df["longitude"].dtype == 'object' or df["longitude"].dtype.name == 'category'):
        df["longitude"] = df["longitude"].str.replace(',', '.').astype('float32')
    if "datahora" in df.columns:
        df['datahora'] = df['datahora'].astype('int64')
    if "linha" in df.columns:
        df['linha'] = pd.Categorical(df['linha']).codes
    if "ordem" in df.columns:
        df['ordem'] = pd.Categorical(df['ordem']).codes
    if "dia_da_semana" in df.columns:
        df['dia_da_semana'] = df['dia_da_semana'].astype('int16')
    if "hora" in df.columns:
        df['hora'] = df['hora'].astype('int16')
    if "diff_timestamp" in df.columns:
        df['diff_timestamp'] = df['diff_timestamp'].astype('float64')
    if "latitude_diff" in df.columns:
        df['latitude_diff'] = df['latitude_diff'].astype('float32')
    if "longitude_diff" in df.columns:
        df['longitude_diff'] = df['longitude_diff'].astype('float32')
    if "distancia" in df.columns:
        df['distancia'] = df['distancia'].astype('float32')
    return df

def convert_to_int_for_merge_pandas(df, col_name):
    df[f'{col_name}_int'] = (df[col_name].astype(str).str.replace(',', '.').astype(float) * 1e6).astype(int)
    return df

def add_date_merge_column(df):
    df['date_merge'] = pd.to_datetime(df['datahora'], unit='ms').dt.date
    return df

def load_and_prepare_data(dados_teste_dir, arquivos, horas_teste, temp_dir):
    temp_files = {
        'train_hist': os.path.join(temp_dir, 'train_hist.parquet'),
        'test_hist': os.path.join(temp_dir, 'test_hist.parquet'),
        'train_datahora': os.path.join(temp_dir, 'train_datahora.parquet'),
        'train_lat_long': os.path.join(temp_dir, 'train_lat_long.parquet'),
        'resposta_datahora': os.path.join(temp_dir, 'resposta_datahora.parquet'),
        'resposta_lat_long': os.path.join(temp_dir, 'resposta_lat_long.parquet')
    }

    if all(os.path.exists(file) for file in temp_files.values()):
        logging.info("Loading preprocessed data from temp files...")
        df_hist_train = cudf.read_parquet(temp_files['train_hist'])
        df_hist_test = cudf.read_parquet(temp_files['test_hist'])
        df_treino_datahora = cudf.read_parquet(temp_files['train_datahora'])
        df_treino_lat_long = cudf.read_parquet(temp_files['train_lat_long'])
        df_resposta_datahora = cudf.read_parquet(temp_files['resposta_datahora'])
        df_resposta_lat_long = cudf.read_parquet(temp_files['resposta_lat_long'])
    else:
        logging.info("Preprocessing data and saving to temp files...")
        if not all(os.path.exists(os.path.join(dados_teste_dir, arquivo)) for arquivo in arquivos):
            logging.error("Nem todos os arquivos necessários foram encontrados. Certifique-se de que todos os arquivos intermediários estão disponíveis.")
            return None, None, None, None, None

        df_hist = pd.read_parquet(os.path.join(dados_teste_dir, 'prepared_data_hist.parquet'))
        df_hist = convert_dtypes_pandas(df_hist)

        df_treino_datahora = pd.read_parquet(os.path.join(dados_teste_dir, 'intermediate_treino_data_datahora.parquet'))
        df_treino_datahora = convert_dtypes_pandas(df_treino_datahora)

        df_treino_lat_long = pd.read_parquet(os.path.join(dados_teste_dir, 'intermediate_treino_data_lat_long.parquet'))
        df_treino_lat_long = convert_dtypes_pandas(df_treino_lat_long)

        df_resposta_datahora = pd.read_parquet(os.path.join(dados_teste_dir, 'intermediate_resposta_data_datahora.parquet'))
        df_resposta_datahora = convert_dtypes_pandas(df_resposta_datahora)

        df_resposta_lat_long = pd.read_parquet(os.path.join(dados_teste_dir, 'intermediate_resposta_data_lat_long.parquet'))
        df_resposta_lat_long = convert_dtypes_pandas(df_resposta_lat_long)

        # Divisão do dataframe de treino e teste
        max_date = pd.to_datetime(df_hist['datahora'].max(), unit='ms')
        test_start_date = max_date - pd.Timedelta(hours=horas_teste)
        
        df_hist['datahora_converted'] = pd.to_datetime(df_hist['datahora'], unit='ms')
        df_hist_train = df_hist[df_hist['datahora_converted'] < test_start_date]
        df_hist_test = df_hist[df_hist['datahora_converted'] >= test_start_date]
        
        # Remover a coluna 'datahora_converted'
        df_hist_train = df_hist_train.drop(columns=['datahora_converted'])
        df_hist_test = df_hist_test.drop(columns=['datahora_converted'])

        # Adicionar colunas auxiliares para o merge
        df_hist_train = add_date_merge_column(df_hist_train)
        df_hist_train = convert_to_int_for_merge_pandas(df_hist_train, 'latitude')
        df_hist_train = convert_to_int_for_merge_pandas(df_hist_train, 'longitude')

        df_treino_lat_long = convert_to_int_for_merge_pandas(df_treino_lat_long, 'latitude')
        df_treino_lat_long = convert_to_int_for_merge_pandas(df_treino_lat_long, 'longitude')    
        df_treino_datahora = add_date_merge_column(df_treino_datahora)

        # Salvar dataframes como parquet
        df_hist_train.to_parquet(temp_files['train_hist'])
        df_hist_test.to_parquet(temp_files['test_hist'])
        df_treino_datahora.to_parquet(temp_files['train_datahora'])
        df_treino_lat_long.to_parquet(temp_files['train_lat_long'])
        df_resposta_datahora.to_parquet(temp_files['resposta_datahora'])
        df_resposta_lat_long.to_parquet(temp_files['resposta_lat_long'])

        del df_hist, df_treino_datahora, df_treino_lat_long, df_resposta_datahora, df_resposta_lat_long
        gc.collect()

        # Recarregar dataframes como cudf.DataFrame
        df_hist_train = cudf.read_parquet(temp_files['train_hist'])
        df_hist_test = cudf.read_parquet(temp_files['test_hist'])
        df_treino_datahora = cudf.read_parquet(temp_files['train_datahora'])
        df_treino_lat_long = cudf.read_parquet(temp_files['train_lat_long'])
        df_resposta_datahora = cudf.read_parquet(temp_files['resposta_datahora'])
        df_resposta_lat_long = cudf.read_parquet(temp_files['resposta_lat_long'])

    return df_hist_train, df_hist_test, df_treino_datahora, df_treino_lat_long, df_resposta_datahora, df_resposta_lat_long

def encode_categorical(df, unique_values):
    mappings = {}
    for col, unique_vals in unique_values.items():
        cat_type = pd.CategoricalDtype(categories=[val for val in unique_vals if pd.notna(val)], ordered=True)
        df[col] = df[col].astype(cat_type).cat.codes
        mappings[col] = {val: code for code, val in enumerate(cat_type.categories)}
    return df, mappings

def ensure_categorical_consistency(train_df, test_df, cat_features):
    for col in cat_features:
        if train_df[col].dtype.name == 'category' and test_df[col].dtype.name == 'category':
            train_cats = set(train_df[col].cat.categories)
            test_cats = set(test_df[col].cat.categories)
            common_cats = train_cats.intersection(test_cats)
            train_df[col] = train_df[col].cat.set_categories(common_cats)
            test_df[col] = test_df[col].cat.set_categories(common_cats)
    return train_df, test_df

def collect_unique_values(dfs, cat_columns):
    combined_unique_values = {col: set() for col in cat_columns}
    for df in dfs:
        for col in cat_columns:
            if col in df.columns:
                combined_unique_values[col].update(df[col].unique().to_pandas())
    return combined_unique_values

def mean_haversine_distance(y_true, y_pred):
    R = 6371  # Radius of the earth in kilometers
    y_true = cp.radians(y_true)
    y_pred = cp.radians(y_pred)
    dlat = y_pred[:, 0] - y_true[:, 0]
    dlon = y_pred[:, 1] - y_true[:, 1]
    a = cp.sin(dlat / 2) ** 2 + cp.cos(y_true[:, 0]) * cp.cos(y_pred[:, 0]) * cp.sin(dlon / 2) ** 2
    c = 2 * cp.arctan2(cp.sqrt(a), cp.sqrt(1 - a))
    distance = R * c
    return cp.mean(distance).get()  # Convert to NumPy array

def run_optimized_models(df, target_cols, save_dir, metric='rmse', combined_unique_values=None, is_location=False):
    df, mappings = encode_categorical(df.to_pandas(), combined_unique_values)
    df = cudf.DataFrame.from_pandas(df)
    X = df.drop(columns=target_cols)
    y = df[target_cols]
    
    X = convert_dtypes_pandas(X.to_pandas())
    y = convert_dtypes_pandas(y.to_pandas())

    def objective_lgbm(trial):
        params = {
            'objective': 'regression',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'device': 'cuda',
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 256),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0)
        }
        
        model = lgb.LGBMRegressor(**params)
        if is_location:
            model = MultiOutputRegressor(model)
        scores = []
        
        for train_index, test_index in tqdm(KFold(n_splits=3).split(X), desc="Cross-validation (LightGBM)"):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            X_train = convert_dtypes_pandas(X_train)
            X_test = convert_dtypes_pandas(X_test)
            y_train = convert_dtypes_pandas(y_train)
            y_test = convert_dtypes_pandas(y_test)
            
            model.fit(X_train, y_train)
            preds = model.predict(X_test)
            if metric == 'rmse':
                score = mean_squared_error(y_test, preds, squared=False)
            elif metric == 'mean_haversine_distance':
                score = mean_haversine_distance(cp.array(y_test), cp.array(preds))
            scores.append(score)
        
        return np.mean(scores)
    
    def objective_xgb(trial):
        params = {
            'verbosity': 0,
            'objective': 'reg:squarederror',
            'booster': 'gbtree',
            'tree_method': 'gpu_hist',
            'gpu_id': 0,
            'eta': trial.suggest_loguniform('eta', 1e-4, 1e-1),
            'max_depth': trial.suggest_int('max_depth', 2, 20),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
            'enable_categorical': True  # Habilitar suporte a categorias
        }
        
        model = xgb.XGBRegressor(**params)
        if is_location:
            model = MultiOutputRegressor(model)
        scores = []
        
        for train_index, test_index in tqdm(KFold(n_splits=3).split(X), desc="Cross-validation (XGBoost)"):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            X_train = convert_dtypes_pandas(X_train)
            X_test = convert_dtypes_pandas(X_test)
            y_train = convert_dtypes_pandas(y_train)
            y_test = convert_dtypes_pandas(y_test)
            
            model.fit(X_train, y_train)
            preds = model.predict(X_test)
            if metric == 'rmse':
                score = mean_squared_error(y_test, preds, squared=False)
            elif metric == 'mean_haversine_distance':
                score = mean_haversine_distance(cp.array(y_test), cp.array(preds))
            scores.append(score)
        
        return np.mean(scores)
    
    # Optuna study for LightGBM
    study_lgbm = optuna.create_study(direction='minimize')
    for _ in tqdm(range(10), desc="LightGBM optimization trials"):
        study_lgbm.optimize(objective_lgbm, n_trials=1)
    
    lgbm_study_path = os.path.join(save_dir, 'lgbm_study.pkl')
    if not os.path.exists(lgbm_study_path):
        with open(lgbm_study_path, 'wb') as f:
            joblib.dump(study_lgbm, f)
    
    # Optuna study for XGBoost
    study_xgb = optuna.create_study(direction='minimize')
    for _ in tqdm(range(10), desc="XGBoost optimization trials"):
        study_xgb.optimize(objective_xgb, n_trials=1)
    
    xgb_study_path = os.path.join(save_dir, 'xgb_study.pkl')
    if not os.path.exists(xgb_study_path):
        with open(xgb_study_path, 'wb') as f:
            joblib.dump(study_xgb, f)

    best_lgbm_trial = study_lgbm.best_trial
    best_xgb_trial = study_xgb.best_trial

    if best_lgbm_trial.value < best_xgb_trial.value:
        return 'lgbm', best_lgbm_trial.params
    else:
        return 'xgb', best_xgb_trial.params

def create_final_model(df, target_cols, best_model_name, best_params, save_dir, model_type):
    final_model_path = os.path.join(save_dir, f'final_model_{model_type}.pkl')
    
    if os.path.exists(final_model_path):
        logging.info(f"Model file {final_model_path} already exists. Skipping model creation.")
        return joblib.load(final_model_path)
    
    df = convert_dtypes_pandas(df.to_pandas())
    X = df.drop(columns=target_cols)
    y = df[target_cols]
    
    X = convert_dtypes_pandas(X)
    y = convert_dtypes_pandas(y)

    if best_model_name == 'lgbm':
        model = lgb.LGBMRegressor(**best_params)
        if model_type == 'lat_long':
            model = MultiOutputRegressor(model)
    else:
        model = xgb.XGBRegressor(**best_params)
        if model_type == 'lat_long':
            model = MultiOutputRegressor(model)

    model.fit(X, y)
    
    joblib.dump(model, final_model_path)

    return model

def create_submission(df_previsto_lat_long, df_resposta_datahora, submissao_lat_long_path):
    previstos_lat_long_short = df_previsto_lat_long[["id", "latitude", "longitude"]].copy()
    # Split the data into manageable chunks to prevent memory issues
    chunk_size = 500000  
    total_rows = previstos_lat_long_short.shape[0]
    chunks = total_rows // chunk_size + 1

    for i in tqdm(range(chunks), desc="Creating submission file"):
        start_row = i * chunk_size
        end_row = min(start_row + chunk_size, total_rows)
        
        df_chunk = previstos_lat_long_short[start_row:end_row].copy()
        df_chunk = df_chunk.merge(
            df_resposta_datahora[['id', 'datahora']].to_pandas(),
            on='id',
            how='inner'
        )
        
        # Convert dtypes to reduce memory usage
        for col in df_chunk.select_dtypes(include=['int64']).columns:
            df_chunk[col] = df_chunk[col].astype('int32')
        for col in df_chunk.select_dtypes(include=['float64']).columns:
            df_chunk[col] = df_chunk[col].astype('float32')
        for col in df_chunk.select_dtypes(include=['object']).columns:
            df_chunk[col] = df_chunk[col].astype('category')

        if i == 0:
            df_chunk.to_csv(submissao_lat_long_path, index=False, mode='w', header=True)
        else:
            df_chunk = df_chunk.drop_duplicates(keep='first')
            df_chunk.to_csv(submissao_lat_long_path, index=False, mode='a', header=False)

    # Save as Parquet
    logging.info("Saving the final submission as Parquet...")
    df_final = pd.read_csv(submissao_lat_long_path)
    df_final.to_parquet(submissao_lat_long_path.replace('.csv', '.parquet'), index=False)

def main():
    logging.basicConfig(level=logging.INFO)
    horas_teste = 5

    df_hist_train, df_hist_test, df_treino_datahora, df_treino_lat_long, df_resposta_datahora, df_resposta_lat_long = load_and_prepare_data(dados_teste_dir, arquivos, horas_teste, temp_dir)
    
    if df_hist_train is None:
        return
    
    cat_features = {'linha': df_hist_train['linha'].unique(), 'ordem': df_hist_train['ordem'].unique()}

    df_hist_train, df_hist_test = ensure_categorical_consistency(df_hist_train, df_hist_test, cat_features)
    
    # Collect unique values for categorical consistency
    combined_unique_values = collect_unique_values(
        [df_hist_train, df_hist_test],
        ['linha', 'ordem']
    )

    # Training and creating model for 'datahora'
    final_model_datahora_path = os.path.join(resultados_dir, 'final_model_datahora.pkl')
    if not os.path.exists(final_model_datahora_path):
        logging.info("Starting optimization for 'datahora' prediction...")
        df_train_datahora = df_hist_train.drop(columns=['dia_da_semana', 'hora', 'diff_timestamp', 'date_merge', 'latitude_int', 'longitude_int'])
        best_model_name, best_params = run_optimized_models(
            df_train_datahora,
            target_cols=['datahora'], 
            save_dir=resultados_dir, 
            metric='rmse', 
            combined_unique_values=combined_unique_values
        )
        best_model_datahora = create_final_model(
            df_train_datahora, 
            target_cols=['datahora'], 
            best_model_name=best_model_name, 
            best_params=best_params, 
            save_dir=resultados_dir, 
            model_type='datahora'
        )
    else:
        logging.info(f"Model file {final_model_datahora_path} already exists. Skipping optimization and loading the model.")
        best_model_datahora = joblib.load(final_model_datahora_path)

    # Training and creating model for 'latitude' and 'longitude'
    final_model_lat_long_path = os.path.join(resultados_dir, 'final_model_lat_long.pkl')
    if not os.path.exists(final_model_lat_long_path):
        logging.info("Starting optimization for 'latitude' and 'longitude' prediction...")
        df_train_lat_long = df_hist_train.drop(columns=['latitude_diff', 'longitude_diff', 'date_merge', 'latitude_int', 'longitude_int'])
        best_model_name, best_params = run_optimized_models(
            df_train_lat_long,  # Drop latitude/longitude-related columns
            target_cols=['latitude', 'longitude'], 
            save_dir=resultados_dir, 
            metric='mean_haversine_distance', 
            combined_unique_values=combined_unique_values,
            is_location=True
        )
        best_model_lat_long = create_final_model(
            df_train_lat_long, 
            target_cols=['latitude', 'longitude'], 
            best_model_name=best_model_name, 
            best_params=best_params, 
            save_dir=resultados_dir, 
            model_type='lat_long'
        )
    else:
        logging.info(f"Model file {final_model_lat_long_path} already exists. Skipping optimization and loading the model.")
        best_model_lat_long = joblib.load(final_model_lat_long_path)

    logging.info("Preparing data for merging...")
    hist_train_datahora = df_hist_train.copy()
    hist_train_lat_long = df_hist_train.copy()

    logging.info("Merging and creating the dataframe df_previsto_datahora...")
    df_previsto_datahora = cudf.merge(
        hist_train_datahora.drop(columns=['datahora']),
        df_treino_lat_long[['ordem', 'linha', 'latitude_int', 'longitude_int', 'id']],
        on=['ordem', 'linha', 'latitude_int', 'longitude_int'],
        how='inner'
    ).drop(columns=['latitude_int', 'longitude_int', 'date_merge'])
    logging.info("Finished merging and creating the dataframe df_previsto_datahora...")

    df_previsto_datahora_id = df_previsto_datahora['id']
    df_previsto_datahora = df_previsto_datahora.drop(columns=['id'])

    df_previsto_datahora = convert_dtypes_pandas(df_previsto_datahora.to_pandas())
    logging.info(f"Input features for datahora prediction: {list(df_previsto_datahora.columns)}")

    df_previsto_datahora = df_previsto_datahora.drop(columns=['dia_da_semana', 'hora', 'diff_timestamp'])

    logging.info("Predicting 'datahora'...")
    df_previsto_datahora['datahora'] = best_model_datahora.predict(df_previsto_datahora)
    df_previsto_datahora['id'] = df_previsto_datahora_id.to_pandas()
    
    logging.info("Creating the submission file to predict datahora...")
    submissao_datahora_path = os.path.join(submissoes_dir, 'submissao_datahora.csv')
    if not os.path.exists(submissao_datahora_path):
        df_submissao_datahora = pd.merge(
            df_previsto_datahora[['id', 'datahora']],
            df_resposta_lat_long[['id', 'latitude', 'longitude']].to_pandas(),
            on='id',
            how='inner'
        )
        df_submissao_datahora.to_csv(submissao_datahora_path, index=False)
    else:
        logging.info(f"{submissao_datahora_path} already exists.")
    logging.info("Finished creating the submission file to predict datahora...")

    logging.info("Merging and creating the dataframe df_previsto_lat_long...")
    df_previsto_lat_long = cudf.merge(
        hist_train_lat_long.drop(columns=['latitude', 'longitude']),
        df_treino_datahora[['ordem', 'linha', 'date_merge', 'id']],
        on=['ordem', 'linha', 'date_merge'],
        how='inner'
    ).drop(columns=['date_merge', 'latitude_int', 'longitude_int'])
    logging.info("Finished merging and creating the dataframe df_previsto_lat_long...")

    df_previsto_lat_long_id = df_previsto_lat_long['id']
    df_previsto_lat_long = df_previsto_lat_long.drop(columns=['id'])

    df_previsto_lat_long = convert_dtypes_pandas(df_previsto_lat_long.to_pandas())
    logging.info(f"Input features for latitude/longitude prediction: {list(df_previsto_lat_long.columns)}")
    
    df_previsto_lat_long = df_previsto_lat_long.drop(columns=['latitude_diff', 'longitude_diff'])

    logging.info("Predicting 'latitude' and 'longitude'...")
    predicted_lat_long = best_model_lat_long.predict(df_previsto_lat_long)
    df_previsto_lat_long['latitude'] = predicted_lat_long[:, 0]
    df_previsto_lat_long['longitude'] = predicted_lat_long[:, 1]
    df_previsto_lat_long['id'] = df_previsto_lat_long_id.to_pandas()

    logging.info("Creating the submission file to predict latitude and longitude...")
    submissao_lat_long_path = os.path.join(submissoes_dir, 'submissao_lat_long.csv')
    if not os.path.exists(submissao_lat_long_path):
        create_submission(df_previsto_lat_long, df_resposta_datahora, submissao_lat_long_path)
    else:
        logging.info(f"{submissao_lat_long_path} already exists.")
    logging.info("Finished creating the submission file to predict latitude and longitude...")
    
    logging.info("Cleaning up...")
    del df_hist_train, df_hist_test, df_treino_datahora, df_treino_lat_long, df_resposta_datahora, df_resposta_lat_long
    del hist_train_datahora, hist_train_lat_long, df_previsto_datahora, df_previsto_lat_long
    gc.collect()

if __name__ == "__main__":
    main()


INFO:root:Loading preprocessed data from temp files...
INFO:root:Model file /mnt/d/POS_GRADUACAO/MESTRADO_PUBLICO/DATA_MINING/TAREFAS/Tarefa3/dados_teste/processados_teste/resultados/final_model_datahora.pkl already exists. Skipping optimization and loading the model.
INFO:root:Model file /mnt/d/POS_GRADUACAO/MESTRADO_PUBLICO/DATA_MINING/TAREFAS/Tarefa3/dados_teste/processados_teste/resultados/final_model_lat_long.pkl already exists. Skipping optimization and loading the model.
INFO:root:Preparing data for merging...
INFO:root:Merging and creating the dataframe df_previsto_datahora...
INFO:root:Finished merging and creating the dataframe df_previsto_datahora...
INFO:root:Input features for datahora prediction: ['latitude', 'longitude', 'labels', 'linha', 'ordem', 'dia_da_semana', 'hora', 'diff_timestamp', 'latitude_diff', 'longitude_diff', 'distancia']
INFO:root:Predicting 'datahora'...
INFO:root:Creating the submission file to predict datahora...
INFO:root:/mnt/d/POS_GRADUACAO/MESTRAD

In [ ]:
import os
import pandas as pd

base_dir = os.getcwd()
dados_teste_dir = os.path.join(base_dir, 'dados_teste', 'processados_teste')
resultados_dir = os.path.join(dados_teste_dir, 'resultados')
path = os.path.join(dados_teste_dir, 'prepared_data_hist.parquet')
path2 = os.path.join(dados_teste_dir, 'intermediate_resposta_data_datahora.parquet')
path3 = os.path.join(dados_teste_dir, 'intermediate_resposta_data_lat_long.parquet')
path4 = os.path.join(dados_teste_dir, 'intermediate_treino_data_datahora.parquet')
path5 = os.path.join(dados_teste_dir, 'intermediate_treino_data_lat_long.parquet')
df_hist = pd.read_parquet(path)
df_resposta_datahora = pd.read_parquet(path2)
df_resposta_lat_long = pd.read_parquet(path3)
df_treino_datahora = pd.read_parquet(path4)
df_treino_lat_long = pd.read_parquet(path5)

In [ ]:
df_hist.linha.nunique()

In [ ]:
df_hist.datahora_converted.min(), df_hist.datahora_converted.max() 

In [ ]:
df_hist.datahora_converted.dt.day.min(), df_hist.datahora_converted.dt.hour.max()

In [ ]:
print("Colunas do arquivo prepared_data_hist.parquet")
print(df_hist.head(2))
print(f"Tamanho do Dataframe: {df_hist.shape[0]}")


In [ ]:
print("Colunas do arquivo intermediate_resposta_data_datahora.parquet")
print(df_resposta_datahora.head(2))
print(f"Tamanho do Dataframe: {df_resposta_datahora.shape[0]}")


In [ ]:
print("Colunas do arquivo intermediate_resposta_data_lat_long.parquet")
print(df_resposta_lat_long.head(2))
print(f"Tamanho do Dataframe: {df_resposta_lat_long.shape[0]}")


In [ ]:
print("Colunas do arquivo intermediate_treino_data_datahora.parquet")
print(df_treino_datahora.head(2))
print(f"Tamanho do Dataframe: {df_treino_datahora.shape[0]}")


In [ ]:
print("Colunas do arquivo intermediate_treino_data_lat_long.parquet")
print(df_treino_lat_long.head(2))
print(f"Tamanho do Dataframe: {df_treino_lat_long.shape[0]}")


In [ ]:
df_treino_datahora.head(10)

In [ ]:
df_hist.dtypes

In [ ]:
print("intermediate_treino_data_datahora.parquet")
df_datahora.head(2)

In [ ]:
print("intermediate_treino_data_lat_long.parquet")
df_lat_long.head(2)

In [ ]:
df_lat_long.isnull().sum()